## Modelo Otimização Viável para Proteção Ambiental

Você foi contratado como consultor por um consórcio intermunicipal para ajudar na conservação de uma serra ameaçada pela fragmentação florestal. Seu objetivo é propor um plano de alocação de áreas para proteção integral, uso sustentável ou reflorestamento, de modo a preservar a biodiversidade, manter os serviços ecossistêmicos e promover o desenvolvimento justo das comunidades locais. A região é dividida em 60 parcelas de terra (com diferentes altitudes, coberturas vegetais e níveis de degradação), mas os dados ambientais disponíveis são parciais, desatualizados e, em muitos casos, baseados em percepções de moradores e gestores locais. Além disso: <br>

- A biodiversidade deve ser respeitada acima de tudo, mas não há consenso sobre quais espécies são mais prioritárias;
- Áreas próximas a nascentes “não podem ser negligenciadas”, segundo líderes comunitários;
- As comunidades exigem que “nenhuma proposta comprometa mais do que o justo das suas possibilidades de sustento”;
- Um relatório técnico recomenda que pelo menos 30% da região seja destinada à proteção integral, mas os prefeitos dizem que isso é "flexível, dependendo da sensatez do plano final".

### Bibliotecas

In [2]:
from pulp import *
import pandas as pd
import numpy as np

### Funções auxiliares

In [3]:
# Função para gerar aleatoriamente o problema 
def random_region(n: int)->pd.DataFrame:
    """
    Método para gerar aleatoriamente um conjunto de dados
    que represente uma região ambiental fictícia.

    Args:
        n (int): Número inteiro que representa as parcelas de terra
        da região
    
    Return:
        dados (pd.DataFrame): A região ambiental aleatória com 
        biodiversidade, degradação, proximidade com nascente e
        importância para o sustento local
    """
    biodiversidade = np.random.randint(0, 10, n)
    degradacao = np.random.random(n)
    nascente = np.random.choice([0, 1], size=n)
    sustento_local = np.random.randint(0, 10, n)

    data = {
        "Biodiversidade": biodiversidade,
        "Degradacao": degradacao,
        "Nascente": nascente,
        "sustento_local": sustento_local
    }

    return pd.DataFrame(data)

In [ ]:
def green_linear_model(dados:pd.DataFrame,
                       n:int,
                       tau:float,
                       K:int,
                       U_max:int):
    """
    Modelo de Otimização Linear para o problema de 
    sustentabilidade da região

    Args:
        dados (pd.DataFrame): Os dados que representam a região
        n (int): Número de parcelas da região
        tau (float): Limiar para considerar "altamente degradado"
        K (int): Mínimo de parcelas altamente degradadas a reflorestar
        U_max (int): Impacto máximo no sustento local 
    """
    prob = LpProblem("GreenModel", LpMaximize)

    # Variáveis de Decisão
    x = [LpVariable(f"x_{i}", cat='Binary') for i in range(n)] # Proteção Ambiental
    y = [LpVariable(f"y_{i}", cat='Binary') for i in range(n)] # Uso sustentável 
    z = [LpVariable(f"z_{i}", cat='Binary') for i in range(n)] # Reflorestamento


    # Parâmetros
    b = dados['Biodiversidade']
    d = dados['Degradacao']
    w = dados['Nascente']
    u = dados['sustento_local']

    # Função-objetivo: maximização da biodiversidade da região
    prob += lpSum(b[i] * x[i] for i in range(n))

    # Restrições
    for i in range(n): 
        if d[i] > tau:
            x[i] == 0 # Se a degradação da parcela i for maior que tau, não preserva biodiversidade
        
        prob += x[i] >= w[i] # Proximidade com nascente não podem ser negligenciadas
        prob += y[i] <= 1 - w[i]  # Uso Sustentável longe de nascentes
        prob += x[i] + y[i] + z[i] == 1 # A parcela i só pode ter apenas um destino
        

    # 30% da região é para proteção ambiental
    prob += lpSum(x[i] for i in range(n)) >= 18   

    # Recuperação mínima de área degradada
    prob += lpSum(z[i] for i in range(n) if d[i] >= tau) >= K

    # Impacto máximo da região i no sustento local
    prob += lpSum(u[i] * x[i] for i in range(n)) <= U_max

    # Resolução
    prob.solve()

    # Resultados
    print("Status:", LpStatus[prob.status])
    print("Biodiversidade protegida:", value(prob.objective))
    print("Parcelas em Proteção Integral:", sum(int(x[i].value()) for i in range(n)))
    print("Parcelas em Uso sustentável:", sum(int(y[i].value()) for i in range(n)))
    print("Parcelas em Reflorestamento:", sum(int(z[i].value()) for i in range(n)))
     

In [30]:
# O conjunto de dados aleatório
database = random_region(n=60)

display(database.head())

,Biodiversidade,Degradacao,Nascente,sustento_local
0,4,0.978724,0,8
1,9,0.782440,1,7
2,0,0.381791,1,6
3,6,0.057223,0,9
4,9,0.669162,1,0


In [72]:
# Resolvendo o problema de maximizar a biodiversidade
green_linear_model(dados=database,
                   n = 60,
                   tau = 0.8,
                   K = 7,
                   U_max= 200)

Status: Optimal
Biodiversidade protegida: 248.0
Parcelas em Proteção Integral: 49
Parcelas em Uso sustentável: 2
Parcelas em Reflorestamento: 9
